In [3]:
import pandas as pd

### Objective:

  | Year | Plant_ID | State | Technologies (Mover, Fuel, Desc) | Production | Demand | Wind Production | Solar Production | (USD)CO2 | (USD)Gas | (USD)coal | 

## Colonnes Year, Month, Plant_ID, Capacity (MW), Technology et State

In [12]:
coal_production = pd.read_csv('treated_data/Production/Coal_Production.csv',sep = ';').drop('Unnamed: 0', axis = 1)

In [13]:
coal_production = coal_production.drop('Technology',axis=1)[['Year','Month','Plant_Id','State', 
                                                            'Mover_Code','Fuel_Code','Production(GWh)']]

## Création colonne Demand

In [14]:
coal_production.Year.unique()

array([2021., 2006., 2007., 2010., 2017., 2016., 2020., 2011., 2013.,
       2012., 2019., 2015., 2009., 2008., 2014., 2018.])

In [16]:
df_demand = pd.read_csv('treated_data/Demand/demand.csv', sep = ';').drop('Unnamed: 0',axis = 1).set_index(['Year','Month'])

In [17]:
panel_i = coal_production.join(df_demand, on=['Year','Month'], how = 'left')

## Création colonnes Wind Production & Solar Production

In [18]:
df_production = pd.read_csv('treated_data/Production/production_complete.csv',sep=';').drop('Unnamed: 0',axis = 1)

In [19]:
df_wind = df_production.where(df_production.Technology == 'Wind').dropna()
df_wind['Wind_Production']=df_wind['Production(GWh)']
df_wind = df_wind.drop(['Production(GWh)','Mover_Code','Fuel_Code','Technology','Plant_Id'],axis=1)

df_solar = df_production.where(df_production.Technology =='Solar').dropna()
df_solar['Solar_Production']=df_solar['Production(GWh)']
df_solar = df_solar.drop(['Production(GWh)','Mover_Code','Fuel_Code','Technology','Plant_Id'],axis=1)

In [20]:
df_wind_join= df_wind.groupby(['Year','Month','State']).sum()
df_solar_join = df_solar.groupby(['Year','Month','State']).sum()

In [21]:
panel_ii_ = panel_i.join(df_wind_join, on=['Year','Month','State'], how = 'left')

In [22]:
panel_ii = panel_ii_.join(df_solar_join, on=['Year','Month','State'], how = 'left')

In [23]:
panel_ii.Wind_Production = panel_ii.Wind_Production.fillna(0)
panel_ii.Solar_Production = panel_ii.Solar_Production.fillna(0)

In [24]:
panel_ii.to_csv('treated_data/Panel/panel_ii.csv',sep=';')

## Création colonnes of prices

In [6]:
panel_ii = pd.read_csv('treated_data/Panel/panel_ii.csv',sep=';').drop('Unnamed: 0',axis=1)